In [1]:
from video_processor import KSoccer

In [3]:
processor = KSoccer(model='yolov8x.pt', source='video', path='inputs/equipo.mp4',ruta_carpeta = 'outputs/test',save = True, scale_factor = 1,depurar=False)
processor.procesar()

Using device: cuda
File opened successfully.

0: 384x640 10 persons, 92.5ms
Speed: 20.4ms preprocess, 92.5ms inference, 515.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 83.8ms
Speed: 0.0ms preprocess, 83.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Imagen del frame guardada como frame_clasificado_1.png
Tiempo de procesamiento del frame 1: 6.3926 segundos

0: 384x640 9 persons, 57.1ms
Speed: 0.0ms preprocess, 57.1ms inference, 15.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.8ms
Speed: 0.0ms preprocess, 61.8ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)
Imagen del frame guardada como frame_clasificado_2.png
Tiempo de procesamiento del frame 2: 0.5252 segundos

0: 384x640 8 persons, 68.0ms
Speed: 2.9ms preprocess, 68.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 82.3ms
Speed: 3.7ms preprocess, 82.3ms inference, 2.0ms p

In [5]:
from video_converter import crear_video_desde_imagenes
ruta_carpeta = 'outputs/test'  # Cambia esto por tu ruta
nombre_video_salida = 'output_videos/e.mp4'  # Nombre del video de salida
crear_video_desde_imagenes(ruta_carpeta, nombre_video_salida)

Video creado correctamente: output_videos/e.mp4
